IMPORT das bibiliotecas de pyspark

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
import pyspark
import json
from pyspark.sql.functions import explode 
from pyspark.sql import functions


<H2> Chamando o dataframe no blob </H2>

In [ ]:
path = "   VALOR       "

#tipo Json
df = spark.read\
        .format('json')\
        .option('inferSchema', True)\
        .load(path)
#tipo CSV
df = spark.read\
        .format('csv')\
        .option('header', True)\
        .option('inferSchema', True)\
        .option('sep', ';')\
        .load(path)
        
# Try 

try:
    df = spark.read\
        .format('csv')\
        .option('header', True)\
        .load(path)
except:
    print("error")
    
else:
    print("Sucesso, Vamos prosseguir")

Tratamentos de Colunas

In [ ]:
#Criando uma função Lambda parar tratar o header da coluna, removendo o espaço vazio e colocando um underscore. 
Nova_Lista= list(map(lambda x: x.replace(" ", "_"), df.columns))
df = d.toDF(*Nova_Lista)
#Abaixo todas as colunas com o nome maisculos
df = df.toDF(*[c.upper() for c in df.columns])

FUNÇÕES PRA TRATAMENTO

In [3]:
#Trocar virgula por ponto OBS: você pode trocar por outras coisas tbm.
def convert_virgula(parametro):
    return df.withColumn(parametro, regexp_replace(parametro, ",", "."))


#Formatar a data 
def formatardata(parametro):
    return df.withColumn(parametro, to_date(parametro, 'dd/MM/yyyy'))

#Converter valor vazio em não informado OBS: Ou algo da sua escolha.
def convertevazio(df1):
    for i in df1.columns:
        df1 = df1.withColumn(i , when(col(i).isNull() | col(i).isin(''),'Não Informado').otherwise(col(i)))
    return df1

#Converter valor nulo em não informado, é o mesmo do de cima...
def converternulo(df1):
    for i in df1.columns:
        df1 = df1.withColumn(i , when(col(i) == "", None).otherwise(col(i)))
    return df1

#Converter Valor null!
def converternulo2(df1):
    for i in df1.columns:
        df1 = df1.na.fill('Não Informado')
    return df1


#Funções de Count
#count nulos
def countnull(df):
    df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c)
               for c in df.columns]).display()

#count não nulos  
def countnotnull(df):
    df.select([count(when(isnan(c) | col(c).isNotNull(), c)).alias(c)
               for c in df.columns]).display()



<H3>Tipo de funções do proprio spark</H3>

In [2]:
#withColumn()
"Tratar valores internos da coluna"
#Eu prefiro utilizar listas, feito o unico que esta situado abaixo.
    df.withColumn('nomedacoluna', when(col('nomedacoluna').isin(unico), 'valorquemudarar').otherwise(col('nomedacoluna')))
    
#withColumnRenamed()
"Trata o nome das colunas"
    df.withColumnRenamed("Antigo-nome", 'Novo-nome')

IndentationError: unexpected indent (385429889.py, line 3)